# Goals
- train predictor on training trusted and untrusted sets 
- use confidence as tampering score

In [1]:
from cupbearer import data, detectors, models, scripts, tasks, utils
from torch.utils.data import DataLoader
import transformers
import torch

/Users/oliverdaniels-koch/miniforge3/envs/cupbearer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Train Measurement Predictor

In [2]:
import os
# uncomment to set cuda visible devices
# os.environ["CUDA_VISIBLE_DEVICES"] = "7"
# os.environ["CUDA_VISIBLE_DEVICES"]

In [3]:
transformer, tokenizer, emb_dim, max_len = models.transformers_hf.load_transformer(
    "pythia-14m"
)
model = models.TamperingPredictionTransformer(
        model=transformer,
        embed_dim=emb_dim
    )
tokenizer = model.set_tokenizer(tokenizer)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
train_data = data.TamperingDataset("diamonds", tokenizer=tokenizer, max_length=max_len, train=True, dataset_len=2)
val_data = data.TamperingDataset("diamonds", tokenizer=tokenizer, max_length=max_len, train=False, dataset_len=2)

In [5]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
num_epochs = 1
total_steps = num_epochs * len(train_loader)
lr = 2e-5
scripts.train_classifier( # NOTE: - paper uses 64 warmup steps, but seems hard
    path=(classifier_path := utils.log_path("logs/tampering/predictor")),
    model=model,
    train_loader=train_loader,
    task="multilabel",
    num_labels=4,
    val_loaders=DataLoader(val_data, batch_size=1024, shuffle=False),
    lr=lr,
    optim_conf={
        "weight_decay": 0.02
    },
    optim_builder=torch.optim.AdamW,
    lr_scheduler_conf={
        "num_warmup_steps": 64,
        "num_training_steps": total_steps
    },
    lr_scheduler_builder=transformers.optimization.get_cosine_schedule_with_warmup,
    max_epochs=num_epochs,
    wandb=True
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenize

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type                           | Params
------------------------------------------------------------------
0 | model          | TamperingPredictionTransformer | 7.6 M 
1 | train_accuracy | MultilabelAccuracy             | 0     
2 | val_accuracy   | ModuleList                     | 0     
3 | test_accuracy  | ModuleList                     | 0     
------------------------------------------------------------------
7.6 M     Trainable params
0         Non-trainable params
7.6 M     Total params
30.517    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/oliverdaniels-koch/miniforge3/envs/cupbearer/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


/Users/oliverdaniels-koch/miniforge3/envs/cupbearer/lib/python3.10/site-packages/torchmetrics/functional/classification/accuracy.py:77: UserWarning: MPS: no support for int64 reduction ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:144.)
  tp = tp.sum(dim=0 if multidim_average == "global" else 1)
/Users/oliverdaniels-koch/miniforge3/envs/cupbearer/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/oliverdaniels-koch/miniforge3/envs/cupbearer/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps 

Epoch 0: 100%|██████████| 1/1 [00:01<00:00,  0.97it/s, v_num=fodv, train/loss=5.860]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:01<00:00,  0.85it/s, v_num=fodv, train/loss=5.860]


{'train/loss': tensor(5.8608),
 'train/acc_step': tensor(0.6250),
 'val/loss': tensor(5.5172),
 'val/acc_step': tensor(0.7500),
 'val/acc_epoch': tensor(0.7500),
 'train/acc_epoch': tensor(0.6250)}